# Understanding Julia's Object Model For Python Developers

- Python is an object-oriented language.
- Julia is not object-oriented.
- It seems converting the Python object code to a Julia counterpart is not straightforward.
- But it is not true.

__________

## A Simple Python Class

- Suppose we have **Vector2D** class that represents a vector in a 2-dimensional Euclidean space.
- **Vector2d** implements a constructor with $x$ and $y$.
- The vector summation operator is implemented by overloading the $+$ operator.
- The implementation is as follows:

In [10]:
class Vector2D:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        return Vector2D(self.x + other.x, self.y + other.y)

- Now suppose the vector $u$ is defined as $u = [1, 2]$.
- The vector $v$ is defined as $v = [3, 4]$.

In [3]:
u = Vector2D(1, 2)
v = Vector2D(3, 4)

- Sum of $u$ and $v$ is obtained by the overloaded `__add__` for **Vector2D** class:

In [4]:
mysum = u + v

- As expected, newly created vector has components of $[1 + 3, 2 + 4] = [4, 6]$:

In [5]:
mysum.x

4

In [6]:
mysum.y

6

## The Julia Counterpart of Vector2D

- Julia has a **struct** keyword for creating user-defined data types.
- Note that a Python class is also a data type.
- Julia supports defining multiple implementations of a single **function**.
- Those instances are called **methods**. 
- Calling a function with different types of arguments is handled by **multiple dispatch**.

------------

- Let's define a new data type called **Vector2D**:

```julia
struct Vector2D
    x
    y
end
```

- Creating an instance is similar to that in the Python code: 

```julia
u = Vector2D(1, 2)
v = Vector2D(3, 4)
```

- The **Vector2D** struct does not encapsulate the summation operator.
- The **Vector2D** data type only includes its data.
- We would implement a constructor, however, we want to keep it simple at this stage.
- $+$ operator defined in the **Base** can be re-implemented for the new type:

```julia
Base.:+(u::Vector2D, v::Vector2D) = Vector2D(u.x + v.x, u.y + v.y)
```

- After creating a new **method** for the function `Base.:+`, we are able to sum $u$ and $v$:

```julia
mysum = u + v
```

- The result is a new **Vector2D** with $x = 4$ and $y = 6$.

- Here is the whole code:

```julia
struct Point2D
    x
    y
end

Base.:+(u::Vector2D, v::Vector2D) = Vector2D(u.x + v.x, u.y + v.y)

u = Point2D(1, 2)
v = Point2D(3, 4)

mysum = u + v
```

## Defining Multiple Constructors

- Python does not support multiple-definition of `__init__`.
- Multiple definition is handled by **default** values of arguments.
- Suppose the default values for $x$ and $y$ are zero. 
- In Python class, we can update the constructor as follows:

In [2]:
class Vector2D:
    
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        return Vector2D(self.x + other.x, self.y + other.y)

- Now, creating an object of **Vector2D** without argumens will end up with a $[0, 0]$ vector.
- Providing a single argument will result $[x, 0]$ or $[0, y]$, respectively:

In [12]:
u = Vector2D()

In [13]:
(u.x, u.y)

(0, 0)

In [14]:
v = Vector2D(5)

In [15]:
(v.x, v.y)

(5, 0)

In [17]:
a = Vector2D(x = 10)

In [18]:
(a.x, a.y)

(10, 0)

In [3]:
b = Vector2D(y = 10)

In [4]:
(b.x, b.y)

(0, 10)

## The Julia Counterpart of Defining Multiple Constructors

- Recall `Vector2D(3, 4)` creates a vector with $x=3$ and $y = 4$. 
- We can define multiple methods with the same name with different number and type of arguments as follows:

```julia
Vector2D() = Vector2D(0,0)
Vector2D(x) = Vector2D(x, x)
Vector2D(; x = 0, y = 0) = Vector2D(x, y)
```

- `Vector2D()` returns a new **Vector2D** with $x=0$ and $y=0$.
- `Vector2D(5)` returns a new **Vector2D** with $x = 5$ and $y = 5$.
- `Vector2D(x = 7, y = 9)` returns a new **Vector2D** with $x = 7$ and $y = 9$. In this case, the order of the arguments is not important and `Vector2D(y = 9, x = 7)` produces the same result.  

## In Short...

- Suppose the Python code is `a.f(x)` where **a** is an object, **f** is the object method, and **x** is the argument.
- The Julia counterpart of this code is `f(a, x)` where **f** is the function, **a** is an instance of a **struct**, and **x** is the argument. If **f** is the reimplementation of a function than it is called a **method** and it is multiple dispacted for the specific types of **a** and **x**.

## Immutability vs. Mutability

- Object members in Python are **mutable**.
- That means at any stage of the program, object members can be changed.
- Julia user-defined data types are **immutable** by default.
- **Mutability** of members should be declared explicitly:

In [19]:
class Vector2D:
    
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y

In [20]:
u = Vector2D(5, 10)
u.x = -20
u.y = -20

- In the code above, members of vector $u$ are mutated.
- The new state of the vector u is $[-20, -20]$.
- Let's take a look at the Julia counterpart:

```julia
struct Vector2D
    x
    y
end

u = Vector2D(5, 10)
u.x = -20
u.y = -20
```

```julia
julia> u.x = -20
ERROR: setfield!: immutable struct of type Vector2D cannot be changed
Stacktrace:
 [1] setproperty!(x::Vector2D, f::Symbol, v::Int64)
   @ Base ./Base.jl:38
 [2] top-level scope
   @ REPL[17]:1

```

- By immutability, Julia compiles the source code into a more efficient binary code.
- If **mutation** is needed, the data type can be implemented by **mutable** keyword:

```julia
mutable struct Vector2D
    x
    y
end

u = Vector2D(5, 10)
u.x = -20
u.y = -20
```

- The new definion of **Vector2D** now allows mutating its member fields.
- In latest versions of Julia, a specific member of a data type can be defined with keyword **mutable** by keeping the remaining fields **immutable**.

## Implementing `__repr__` and `__str__`

- Python provides some methods for representing the objects.
- `__repr__` and `__str__` are special functions for representing the object in REPL and obtaining a `str` representation of object, respectively.
- Suppose the **Vector2D** is reimplemented with these methods:

In [23]:
class Vector2D:
    
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Vector2D({self.x}, {self.y})"
    
    def __str__(self):
        return f"Vector2D({self.x}, {self.y})"

- When we create an instance of this object, we can *pretty-print* it by

In [24]:
u = Vector2D(5, 7)
print(u)

Vector2D(5, 7)


In [25]:
u

Vector2D(5, 7)

## The Julia Counterparts of `__repr__` and `__str__`

- Julia provides default *printer*s for user-defined data types:

```julia
mutable struct Vector2D
    x
    y
end

u = Vector2D(5, 10)

julia> u
Vector2D(5, 10)
```

- `show` function can be redefined to create a method for printing **Vector2D**.
- This function is defined in the **Base** module of Julia.

```julia 
Base.show(io::IO, v::Vector2D) = println(io, "-> Vector2D($(v.x), $(v.y))")
```

- Now, any object in type of **Vector2D** is represented in a new form:

```julia
julia> u
-> Vector2D(5, 10)
```

- `string` function converts its argument into string data type.
- This function works for object in many types (by multiple dispatch).
- Let's implement a method for this function for converting **Vector2D** to string.

```julia
julia> Base.string(v::Vector2D) = "Vector2D[x = $(v.x), y = $(v.y)]"

julia> string(u)
"Vector2D[x = 5, y = 10]"
```

- `convert` defined in **Base** module can also be redefined to convert **Vector2D** to String.

```julia
julia> Base.convert(::Type{String}, v::Vector2D) = "Vector2D[x = $(v.x), y = $(v.y)]"

julia> convert(String, u)
"Vector2D[x = 5, y = 10]"
```

## Type Hints

- Python's type system is dynamic (so does Julia's, but works in a different way!).
- `def f(x, y)` accepts $x$ and $y$ of any types.
- Type hints defines types for function arguments.
- These type hintings doesn't prevent passing of arguments in illegal (or unsupported) types.
- Suppose we redefine **Vector2D** with **float** data type:

In [33]:
class Vector2D:
    
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Vector2D({self.x}, {self.y})"

In [34]:
u = Vector2D(5, 6)

In [35]:
u

Vector2D(5, 6)

In [36]:
v = Vector2D(5.0, 6.0)

In [37]:
v

Vector2D(5.0, 6.0)

- Type hints don't have an effect here (at runtime).
- Object creation for int types are also possible.
- However, it is a useful feature for linting, documenting, and auto-completion of IDEs.

## Julia Counterpart of Python's Type Hints

- There is not a 1-1 correspondence between Python's Type Hints and Julia's type system.
- Method invocation in Julia is based on real types.
- Suppose **Vector2D** is defined for float data type in Julia

```julia
struct Vector2D
    x::Float64
    y::Float64
end 

```

- Let's create an instance with $x=10$ and $y = 5$.

```julia
julia> Vector2D(10.0, 5.0)
Vector2D(10.0, 5.0)
```

- Object creation using **Complex** is not possible, because, user-defined type **Vector2D** is not defined for **Complex** data type.

```julia
julia> Vector2D(3, 7 + 2im)
ERROR: InexactError: Float64(7 + 2im)
Stacktrace:
 [1] Real
   @ ./complex.jl:44 [inlined]
 [2] convert
   @ ./number.jl:7 [inlined]
 [3] Vector2D(x::Int64, y::Complex{Int64})
   @ Main ./REPL[1]:2
 [4] top-level scope
   @ REPL[7]:1
```

- Function/Method definitons can be restricted for a small set of type of arguments:

```julia
julia> function f(x::Int64, y::Int64)::Int64
           if x < y
               return x
           else
               return x + y
           end
       end 
f (generic function with 1 method)
```

- Calls like `f(2.3, 6)` is not allowed: 

```julia
ulia> f(2.3, 6)
ERROR: MethodError: no method matching f(::Float64, ::Int64)

Closest candidates are:
  f(::Int64, ::Int64)
   @ Main REPL[8]:1

Stacktrace:
 [1] top-level scope
   @ REPL[9]:1

```

## Finally...

- Suppose you decide to migrate your Python codebase to Julia.
- Implement **struct**s for each Python class in Julia.
- If object members are mutable, use **mutable struct**.
- Define new methods (or operators) that work with these new types.